# Brain Tumor Segmentation with U-Net

This notebook trains and evaluates a U-Net model for brain tumor segmentation using TensorFlow 2.0.

## Pipeline Steps:
1. Mount Google Drive and extract dataset
2. Clone the GitHub repository
3. Run training and evaluation pipeline

## Step 1: Mount Google Drive and Extract Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Extract the brain tumor dataset
# Make sure to upload your dataset zip file to Google Drive first
# Update the path below to point to your dataset zip file
!unzip /content/drive/MyDrive/brain_tumor_dataset.zip -d /content/dataset

## Step 2: Clone the GitHub Repository

In [ ]:
# Clone the repository
!git clone https://github.com/Innocentaman/arsvd.git

# Navigate to the project directory
%cd arsvd

# List the contents to verify
!ls -la

## Step 3: Install Dependencies and Run Pipeline

In [ ]:
# Install required packages
# TensorFlow 2.19.0 is already installed in Colab, so we'll use that version
# We just need to ensure other dependencies are installed
!pip install opencv-python numpy pandas scikit-learn tqdm

In [ ]:
# Run the complete training and evaluation pipeline
# This will:
# 1. Train the U-Net model on the dataset
# 2. Evaluate the model on test data
# 3. Save results, metrics, and visualizations

!python run_pipeline.py \
  --data_root /content/dataset/data \
  --out_dir ./artifacts \
  --epochs 50 \
  --batch_size 16 \
  --lr 1e-4 \
  --img_size 256 \
  --patience 20 \
  --lr_patience 5 \
  --seed 42

## View Results

In [ ]:
# Display training metrics
import pandas as pd
import matplotlib.pyplot as plt

# Load training history
log_df = pd.read_csv('./artifacts/log.csv')
print("Training History:")
print(log_df.tail())

# Plot training curves
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(log_df['epoch'], log_df['loss'], label='Train Loss')
plt.plot(log_df['epoch'], log_df['val_loss'], label='Val Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(log_df['epoch'], log_df['dice_coef'], label='Train Dice Coef')
plt.plot(log_df['epoch'], log_df['val_dice_coef'], label='Val Dice Coef')
plt.title('Training and Validation Dice Coefficient')
plt.xlabel('Epoch')
plt.ylabel('Dice Coefficient')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Display test metrics
score_df = pd.read_csv('./artifacts/score.csv')
print("\nTest Results:")
print(f"Average F1 Score: {score_df['F1'].mean():.4f}")
print(f"Average Jaccard Index: {score_df['Jaccard'].mean():.4f}")
print(f"Average Recall: {score_df['Recall'].mean():.4f}")
print(f"Average Precision: {score_df['Precision'].mean():.4f}")

print("\nPer-image results:")
print(score_df.head(10))

In [ ]:
# Display some prediction results
from IPython.display import Image, display
import os

results_dir = './artifacts/results'
result_images = sorted(os.listdir(results_dir))[:5]  # Display first 5 results

for img_name in result_images:
    img_path = os.path.join(results_dir, img_name)
    print(f"\n{img_name}")
    display(Image(filename=img_path, width=800))

In [ ]:
# Copy results to Google Drive for permanent storage
!cp -r ./artifacts /content/drive/MyDrive/brain_tumor_segmentation_results
print("Results saved to: /content/drive/MyDrive/brain_tumor_segmentation_results")